In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from wdtools import *

In [ ]:
# this will take about 9 to 12 minutes
start = time.time()
all_taxlot = combine_taxlot()
end = time.time()
print(f'it took {round((end - start)/60, 0)} minutes to complete')

In [4]:
setID = 'Set002'

In [8]:
nm_to_add=1420

In [5]:
wd = combine_wd_tables(setID=setID, nm_to_add=nm_to_add)

In [6]:
wd.shape

(723, 25)

In [7]:
df = combined_reindexed_data(setID=setID, nm_to_add=nm_to_add)

In [9]:
start = time.time()
gdf = match_wd_data_with_taxlot(df=df, setID=setID, all_taxlot=all_taxlot, export=True)
end = time.time()
print(f'it took {round((end - start)/60, 0)} minutes to complete')

it took 8.0 minutes to complete


In [10]:
unmatched_df = report_unmatched(gdf=gdf, setID=setID, nm_to_add=1420, mute = False)

it is about 10.24% of data in the original 723 records unmatched
there are 1 records (0.14% of the original records) without parcel id


In [11]:
unmatched_df.to_csv(os.path.join(inpath + '\\output\\to_review\\', f'unmatched_df_{setID}.csv'), index=False) 

In [12]:
r1_df, r2_df = split_unmatched_df(unmatched_df, ml='N', setID=setID)

In [13]:
# this might take a while (8 minutes)
start = time.time()
rev_r2 = review_unmatched_df_r2(df=r2_df, taxlot=all_taxlot, setID=setID, ml='N', export=True)
end = time.time()
print(f'it took {round((end - start)/60, 0)} minutes to complete')

reviewing WD2017-0498
all lots are matched...
corrected trsqq...
reviewing WD2021-0013
all lots are matched...
corrected trsqq...
reviewing WD2020-0620
all lots are matched...
corrected trsqq...
reviewing WD2021-0285
all lots are matched...
corrected trsqq...
reviewing WD2021-0651
there is not any matched lot, need to review trsqq, the close-match is 03N27E
lots to check: ['1702'], and lots to compare: ['106', '107', '1100', '1102', '1103', 'T00000100', 'T00000101', 'T00000104', 'T00000105', 'T00000106', 'T00000107', 'T00000108', 'T00000200', 'T00000300', 'T00000400', 'T00000700', 'T00000800', 'T00000900', 'T00001100', 'T00001101', 'T00001200', 'T00001204', 'T00001300', 'T00001400', 'T00001401', 'T00001700', 'T00001701', 'T00001702', 'ROADS', '100', '300', '301', '500', '600', '800', '900', '1000', '1200', '1300', '1400', '1500', '1600', '1700', '1800', '1900', '2000', '2100', '2200', '2300', '2400', '2500', '2600', '2700', '2800', '2900', '3000', '3100', '3200', '3300', '3400', '3500'

In [17]:
wdID = 'WD2021-0422'
url = rev_r2.loc[rev_r2.wetdet_delin_number == wdID, 'DecisionLink'].values[0]
if str(url) == 'nan':
    print('Decision link is not available')
else:
    webbrowser.open(url)

Decision link is not available


In [18]:
rev_r2.loc[rev_r2.wetdet_delin_number == wdID, ['correct_type', 'correction', 'cor_trsqq']]

,correct_type,correction,cor_trsqq
1144,QQ,from 00 to D0,39S20E04D0


In [20]:
check_review_notes_r2n(wdID = wdID, df = rev_r2)

Decision link is not available


,correct_type,correction,cor_trsqq
1144,QQ,from 00 to D0,39S20E04D0


In [16]:
#review_loop(df=rev_r2)

In [16]:
r1_df.wetdet_delin_number.unique()

array(['WD2021-0250'], dtype=object)

In [17]:
review_loop_r1(df=r1_df)

WD2021-0250
   county      trsqq            parcel_id   latitude   longitude  record_ID  \
56  Baker  08S37ED02  3000,3100,3500,3601  44.831504 -118.198506       1477   
57  Baker  08S37ED05     101,300,400,2000  44.831727 -118.199277       1478   

   notes  
56        
57        
Press 'p' to pause or any other key to continue...c


In [18]:
setID

'Set002'

In [17]:
cor_r2 = correct_unmatched(r2_df, setID, s='r2', ml='N', export=True)

In [18]:
cor_r2_2 = update_unmatched_df_r2(r2_df, setID, ml='N', export=True)

In [19]:
rev_df = reindex_data(cor_r2_2)

In [20]:
# make sure the matched data before update is correct
matched = match_wd_data_with_taxlot(rev_df, setID, all_taxlot, export=True, update=True)

In [21]:
unmatched_df = report_unmatched(gdf=matched, setID=setID, nm_to_add=1420, mute = False)

it is about 8.02% of data in the original 723 records unmatched
there are 1 records (0.14% of the original records) without parcel id


In [22]:
missedID = [rID for rID in cor_r2_2.record_ID.values if rID not in matched.record_ID.values]

In [23]:
len(missedID)

12

In [24]:
matched_toReview = matched[matched.notes.notnull()] 

In [25]:
len(matched_toReview.wdID.unique())

140

In [26]:
wd_toReview = wd[wd.wetdet_delin_number.isin(matched_toReview.wdID.unique())]

In [27]:
outpath = r'L:\NaturalResources\Wetlands\Local Wetland Inventory\WAPO\EPA_2022_Tasks\Task 1 WD Mapping\output'

In [28]:
wd_toReview.to_csv(outpath + f'\\to_review\\partial_matched_{setID}.csv', index=False)

In [42]:
unmatched_df.shape

(62, 25)

In [32]:
unmatched_df.columns

Index(['wetdet_delin_number', 'trsqq', 'parcel_id', 'address_location_desc',
       'city', 'county', 'site_name', 'site_desc', 'latitude', 'longitude',
       'Coord-Source', 'DocumentName', 'DecisionLink', 'is_batch_file',
       'status_name', 'received_date', 'response_date',
       'reissuance_response_date', 'project_id', 'site_id', 'record_ID',
       'notes', 'recyear', 'IDyear', 'missinglot'],
      dtype='object')

In [43]:
setgdf = gpd.read_file(os.path.join(inpath, 'GIS', 'Join_Statewide.gdb'), layer=f'WD_{setID}_Combined')